In [2]:
import tweepy
import json 
import csv
from os import remove
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

RuntimeError: The current Numpy installation ('D:\\Users\\leand\\anaconda3\\envs\\simulacion\\lib\\site-packages\\numpy\\__init__.py') fails to pass a sanity check due to a bug in the windows runtime. See this issue for more information: https://tinyurl.com/y3dm3h86

### Autenticación de usuario para la app de twitter

In [ ]:
consumer_key = 'FsNkq2SyJl469EAnCJlzQjTbW'
consumer_secret = 'LdBzdUNzoM9dcz4Oa2BOCp8Lp6IqdvRk3khvxYRdwnWQdxnDPl'
access_token ='1339939296611069957-g6C97ncUsHoBAj5y8mH6XOegM90jv5' 
access_token_secret = 'J9hkEtJaNC1Doxe2PhE6ymT2l83x77zFUjRAkwrW0jiWk'

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)

auth.set_access_token(access_token,access_token_secret)
#OBTENER MI INFORMACION
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) 

### Creación de un array con los candidatos a precidencia de la república del Ecuador
#### Estos son los nombres que usan los candidatos en la red social Twitter 


In [ ]:
candidatos = ['LassoGuillermo','AbAlvaroNoboa', 'ecuarauz' , 'yakuperezg','juanfervelasco'  ]
#candidatos_Asamblea = ['karlabeltrang','ceciline1','LourdesCuestaO', 'AndreGonzalezBe']
candidatos

### Crear un objeto candidato  y este almacenar en un array de objetos candidato

In [ ]:
row_data=[]

### Obtenemos la información relevantes para la regresión de la cuenta del candidato

In [ ]:
remove("candidatos.csv")


name, user, location, follower, fecha, id_tweet, text, menciones, likes, hashtags, share = '','', '' ,'','','','','','','',''
rows  = [['Nombre', 'usuario', 'locacion', 'followers Twitter', 'fecha tweet',
          'id tweet', 'contenido','menciones' , 'likes', 'hashtags', 'veces compartido']]
contador_filas =0
for candidato in candidatos:
    data = api.get_user(candidato)
    print('DATOS DEL CANDIDATO: '+ data._json['name'])
    print('usuario  => ' +data._json['screen_name'])
    print('Ubicación => '+data._json['location'])
    print('Seguidores en Twitter => '+ str(data._json['followers_count']))
    #print(type(data._json['entities']))
    #print(type(data._json['status']))
    diccionario =data._json['entities'] 
    lista=[]
   
    for link in diccionario :  #recorremos 
        valor = diccionario[link]
        for vrd_valor in valor:
            lista = [lista,valor[vrd_valor]]
            
    new_lista = lista[0]
    listaA=new_lista[1]
    i=0;
    link_candidato=''
    for dia in listaA:
        link =dia
        for links in link:
            i=i+1
            if i == 2:
                link_candidato =link[links]
                
            #print(link[links])

    print(link_candidato)
    print('TWEETS DEL CANDIDATO')
    name, user, location, followers = data._json['name'],data._json['screen_name'],data._json['location'],data._json['followers_count']
    
    for tweet in tweepy.Cursor(api.user_timeline, screen_name=candidato, tweet_mode = 'extend').items(25):
        #print(json.dumps(tweet._json, indent=2))
        #print('DESCRICPCIÓN TWEET:' )
        #print('PUBLICADO EL   => ' +str(tweet._json['created_at']))
        #print('ID tweet  => ' +str(tweet._json['id_str']))
        #print('TEXTO  => ' +str(tweet._json['text']))
        #print('RETWITEADO  => ' +str(tweet._json['retweet_count']) + ' veces')
        #print("LIKE's => " +str(tweet._json['favorite_count']) )
        #print('Lugar  => ' +str(tweet._json['entities']))
        #print(tweet._json['entities'])
        diccionario =tweet._json['entities'] 
        hashtags = diccionario['hashtags']
        menciones = diccionario['user_mentions']
        i=0;
        j=0;
        #print('HASHTAGS:')
        users_mnc=''
        hastags=''
        for dia in hashtags:
            hst = dia
            for links in hst:

                i=i+1
                if i == 1:
                    hastags =hst[links]
                    #print(hastags)

        #print('USERS MECIONADOS:')
        for m in menciones:
            mnc = m
           #print(mnc)
            for m_user in mnc:
                j=j+1
                if j == 1:
                    users_mnc=mnc[m_user]
                    #print(mnc[m_user])
        fecha, id_tweet, text, menciones, likes, hashtags, share=tweet._json['created_at'],users_mnc,tweet._json['id_str'],tweet._json['text'].encode("utf-8"),tweet._json['favorite_count'],hastags,tweet._json['retweet_count']       
        single_row=[name, user, location, follower, fecha, id_tweet, text, menciones, likes, hashtags, share]
        rows.append(single_row)
        

#for datos in row_data:
    #print(datos)

with open('candidatos.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(rows)